In [1]:
import neptune


NEPTUNE_API_TOKEN = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxMGM5ZDhiMy1kOTlhLTRlMTAtOGFlYy1hOTQzMDE1YjZlNjcifQ=='


## Remove some mistakenly made fields from a test Run

In [2]:
run_id = 'RCC-1296'
run = neptune.init_run(project='revivemed/RCC',
                        api_token=NEPTUNE_API_TOKEN,
                        with_id=run_id,
                        capture_stdout=False,
                        capture_stderr=False,
                        capture_hardware_metrics=False)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/RCC/e/RCC-1296


In [6]:
run_structure = run.get_structure()

remov_keys = []
for key, value in run_structure.items():
    # print(key, value)

    if 'MSKCC Train AUROC' in key:
        remov_keys.append(key)
    if 'MSKCC Val AUROC' in key:
        remov_keys.append(key)

In [8]:
print(remov_keys)
for key in remov_keys:
    del run[key]

In [9]:
run.stop()

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/RCC/e/RCC-1296/metadata


## Remove uncessary fields from larger set of runs

In [13]:
project = neptune.init_project(
    project='revivemed/RCC',
    mode="read-only",
    api_token=NEPTUNE_API_TOKEN
)

runs_table_df = project.fetch_runs_table(state='inactive').to_pandas()

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/RCC/


Fetching table...: 0 [00:00, ?/s]

[neptune] [warning] NeptuneWarning: Date parsing failed. The date format is incorrect. Returning as string instead of datetime.


In [19]:
# Select runs with any of the specific tags
run_id_list = []
tag_list = ['april2','king','pretrain_plots','jack','queen','april03_pareto']
for row in runs_table_df.iterrows():
    tags = row[1]['sys/tags'].split(',')
    for tag in tags:
        if tag in tag_list:
            run_id_list.append(row[1]['sys/id'])
            break
        


In [18]:
len(run_id_list)

34

In [21]:
# fields to remove
rem_fields = ['finetune_mskcc_','finetune_MSKCC','randinit_MSKCC','randinit_mskcc_','summary']
for ii in range(5):
    rem_fields.append('finetune_mskcc_'+str(ii))
    rem_fields.append('randinit_mskcc_'+str(ii))

print(rem_fields)

['finetune_mskcc_', 'finetune_MSKCC', 'randinit_MSKCC', 'summary', 'finetune_mskcc_0', 'randinit_mskcc_0', 'finetune_mskcc_1', 'randinit_mskcc_1', 'finetune_mskcc_2', 'randinit_mskcc_2', 'finetune_mskcc_3', 'randinit_mskcc_3', 'finetune_mskcc_4', 'randinit_mskcc_4']


In [22]:
for run_id in run_id_list:
    print(run_id)
    run = neptune.init_run(project='revivemed/RCC',
                        api_token=NEPTUNE_API_TOKEN,
                        with_id=run_id,
                        capture_stdout=False,
                        capture_stderr=False,
                        capture_hardware_metrics=False)
    
    run_structure = run.get_structure()

    remov_keys = []
    for key, value in run_structure.items():
        # print(key, value)

        for field in rem_fields:
            if key == field:
                remov_keys.append(key)
                break


    print('deleting the following keys:', remov_keys)
    for key in remov_keys:
        del run[key]

    run.stop()
    break

RCC-1603
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/RCC/e/RCC-1603
deleting the following keys: ['finetune_MSKCC', 'finetune_mskcc_0', 'finetune_mskcc_1', 'finetune_mskcc_2', 'finetune_mskcc_3', 'finetune_mskcc_4', 'randinit_MSKCC', 'randinit_mskcc_0', 'randinit_mskcc_1', 'randinit_mskcc_2', 'randinit_mskcc_3', 'randinit_mskcc_4', 'summary']
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 1685 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] Still waiting for the remaining 1685 operations (0.00% done). Please wait.
[neptune] [info   ] Still waiting for the remaining 1685 operations (0.00% done). Please wait.
[neptune] [warning] Experiencing connection interruptions. Will try to reestablish communication with Neptune. Internal exception was: HTTPTooManyRequests
[neptune] [info   ] Still waiting for the rem